<a href="https://colab.research.google.com/github/Swagat1342/Enhanced-Stock-Prediction-System-with-Risk-Management/blob/main/finance_forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install numpy pandas yfinance scikit-learn tensorflow plotly

In [ ]:
pip install alpha_vantage finnhub-python polygon-api-client

In [ ]:
pip install --upgrade yfinance


In [ ]:
pip install yfinance ta pandas numpy plotly


In [2]:
import warnings
warnings.filterwarnings('ignore')

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit


In [3]:
INDIAN_STOCKS = {
    'NIFTY_50': {
        'RELIANCE.NS': 'Reliance Industries', 'TCS.NS': 'Tata Consultancy Services',
        'HDFCBANK.NS': 'HDFC Bank', 'INFY.NS': 'Infosys', 'ICICIBANK.NS': 'ICICI Bank',
        'HINDUNILVR.NS': 'Hindustan Unilever', 'ITC.NS': 'ITC Limited', 'SBIN.NS': 'State Bank of India',
        'BHARTIARTL.NS': 'Bharti Airtel', 'KOTAKBANK.NS': 'Kotak Mahindra Bank', 'LT.NS': 'Larsen & Toubro',
        'AXISBANK.NS': 'Axis Bank', 'BAJFINANCE.NS': 'Bajaj Finance', 'ASIANPAINT.NS': 'Asian Paints',
        'MARUTI.NS': 'Maruti Suzuki', 'HCLTECH.NS': 'HCL Technologies', 'WIPRO.NS': 'Wipro',
        'ULTRACEMCO.NS': 'UltraTech Cement', 'TITAN.NS': 'Titan Company', 'NESTLEIND.NS': 'Nestle India',
        'SUNPHARMA.NS': 'Sun Pharmaceutical', 'TECHM.NS': 'Tech Mahindra', 'POWERGRID.NS': 'Power Grid Corporation',
        'NTPC.NS': 'NTPC Limited', 'M&M.NS': 'Mahindra & Mahindra', 'TATAMOTORS.NS': 'Tata Motors',
        'BAJAJFINSV.NS': 'Bajaj Finserv', 'ADANIPORTS.NS': 'Adani Ports', 'COALINDIA.NS': 'Coal India',
        'ONGC.NS': 'ONGC', 'TATASTEEL.NS': 'Tata Steel', 'DIVISLAB.NS': 'Divi\'s Laboratories',
        'DRREDDY.NS': 'Dr. Reddy\'s Laboratories', 'GRASIM.NS': 'Grasim Industries', 'HINDALCO.NS': 'Hindalco Industries',
        'INDUSINDBK.NS': 'IndusInd Bank', 'JSWSTEEL.NS': 'JSW Steel', 'EICHERMOT.NS': 'Eicher Motors',
        'HEROMOTOCO.NS': 'Hero MotoCorp', 'CIPLA.NS': 'Cipla', 'BRITANNIA.NS': 'Britannia Industries',
        'SHREECEM.NS': 'Shree Cement', 'UPL.NS': 'UPL Limited', 'APOLLOHOSP.NS': 'Apollo Hospitals',
        'BPCL.NS': 'Bharat Petroleum', 'ADANIENT.NS': 'Adani Enterprises', 'BAJAJ-AUTO.NS': 'Bajaj Auto',
        'TATACONSUM.NS': 'Tata Consumer Products', 'SBILIFE.NS': 'SBI Life Insurance', 'HDFCLIFE.NS': 'HDFC Life Insurance',
    },
    'BANKING': {
        'HDFCBANK.NS': 'HDFC Bank', 'ICICIBANK.NS': 'ICICI Bank', 'SBIN.NS': 'State Bank of India',
        'KOTAKBANK.NS': 'Kotak Mahindra Bank', 'AXISBANK.NS': 'Axis Bank', 'INDUSINDBK.NS': 'IndusInd Bank',
        'BANDHANBNK.NS': 'Bandhan Bank', 'FEDERALBNK.NS': 'Federal Bank', 'IDFCFIRSTB.NS': 'IDFC First Bank',
        'PNB.NS': 'Punjab National Bank', 'BANKBARODA.NS': 'Bank of Baroda', 'CANBK.NS': 'Canara Bank',
    },
    'IT': {
        'TCS.NS': 'Tata Consultancy Services', 'INFY.NS': 'Infosys', 'HCLTECH.NS': 'HCL Technologies',
        'WIPRO.NS': 'Wipro', 'TECHM.NS': 'Tech Mahindra', 'LTIM.NS': 'LTIMindtree',
        'PERSISTENT.NS': 'Persistent Systems', 'COFORGE.NS': 'Coforge', 'MPHASIS.NS': 'Mphasis',
    },
    'PHARMA': {
        'SUNPHARMA.NS': 'Sun Pharmaceutical', 'DIVISLAB.NS': 'Divi\'s Laboratories', 'DRREDDY.NS': 'Dr. Reddy\'s Laboratories',
        'CIPLA.NS': 'Cipla', 'AUROPHARMA.NS': 'Aurobindo Pharma', 'LUPIN.NS': 'Lupin',
        'BIOCON.NS': 'Biocon', 'TORNTPHARM.NS': 'Torrent Pharmaceuticals', 'ALKEM.NS': 'Alkem Laboratories',
    },
    'AUTO': {
        'MARUTI.NS': 'Maruti Suzuki', 'TATAMOTORS.NS': 'Tata Motors', 'M&M.NS': 'Mahindra & Mahindra',
        'EICHERMOT.NS': 'Eicher Motors', 'HEROMOTOCO.NS': 'Hero MotoCorp', 'BAJAJ-AUTO.NS': 'Bajaj Auto',
        'TVSMOTOR.NS': 'TVS Motor', 'ASHOKLEY.NS': 'Ashok Leyland', 'MOTHERSON.NS': 'Motherson Sumi',
    },
    'FMCG': {
        'HINDUNILVR.NS': 'Hindustan Unilever', 'ITC.NS': 'ITC Limited', 'NESTLEIND.NS': 'Nestle India',
        'BRITANNIA.NS': 'Britannia Industries', 'DABUR.NS': 'Dabur India', 'MARICO.NS': 'Marico',
        'GODREJCP.NS': 'Godrej Consumer Products', 'COLPAL.NS': 'Colgate-Palmolive', 'TATACONSUM.NS': 'Tata Consumer Products',
    },
    'ENERGY': {
        'RELIANCE.NS': 'Reliance Industries', 'ONGC.NS': 'ONGC', 'BPCL.NS': 'Bharat Petroleum',
        'IOC.NS': 'Indian Oil Corporation', 'NTPC.NS': 'NTPC', 'POWERGRID.NS': 'Power Grid Corporation',
        'COALINDIA.NS': 'Coal India', 'GAIL.NS': 'GAIL India', 'ADANIGREEN.NS': 'Adani Green Energy',
        'TATAPOWER.NS': 'Tata Power',
    },
    'METALS': {
        'TATASTEEL.NS': 'Tata Steel', 'HINDALCO.NS': 'Hindalco Industries', 'JSWSTEEL.NS': 'JSW Steel',
        'COALINDIA.NS': 'Coal India', 'VEDL.NS': 'Vedanta', 'SAIL.NS': 'SAIL',
        'NMDC.NS': 'NMDC', 'HINDZINC.NS': 'Hindustan Zinc',
    },
    'INFRA': {
        'LT.NS': 'Larsen & Toubro', 'ADANIPORTS.NS': 'Adani Ports', 'ULTRACEMCO.NS': 'UltraTech Cement',
        'SHREECEM.NS': 'Shree Cement', 'GRASIM.NS': 'Grasim Industries', 'AMBUJACEM.NS': 'Ambuja Cements',
        'ACC.NS': 'ACC Limited',
    },
    'TELECOM': {
        'BHARTIARTL.NS': 'Bharti Airtel', 'IDEA.NS': 'Vodafone Idea',
    },
}

ALL_INDIAN_STOCKS = {}
for sector_stocks in INDIAN_STOCKS.values():
    ALL_INDIAN_STOCKS.update(sector_stocks)

print(f"✅ Loaded {len(ALL_INDIAN_STOCKS)} Indian stocks across {len(INDIAN_STOCKS)} sectors")


✅ Loaded 83 Indian stocks across 10 sectors


In [4]:
def search_stock(query):
    """🔍 Search for stocks by name or ticker, or list sector stocks"""
    query = query.upper()

    # First check if it's a sector name
    if query in INDIAN_STOCKS:
        print(f"\n{'='*70}")
        print(f"📊 '{query}' is a SECTOR! Showing all {len(INDIAN_STOCKS[query])} stocks in this sector:")
        print(f"{'='*70}")
        print(f"{'Ticker':<20} {'Company Name'}")
        print("-" * 70)
        for ticker, name in INDIAN_STOCKS[query].items():
            print(f"{ticker:<20} {name}")
        print("="*70)
        return list(INDIAN_STOCKS[query].items())

    # Search for stocks by name or ticker
    results = []
    for ticker, name in ALL_INDIAN_STOCKS.items():
        if query in ticker.upper() or query in name.upper():
            results.append((ticker, name))

    if results:
        print(f"\n{'='*70}")
        print(f"🔍 SEARCH RESULTS FOR: '{query}' - Found {len(results)} matches")
        print(f"{'='*70}")
        print(f"{'Ticker':<20} {'Company Name'}")
        print("-" * 70)
        for ticker, name in results:
            print(f"{ticker:<20} {name}")
        print("="*70)
    else:
        print(f"\n❌ No stocks found matching '{query}'")
        print("\n💡 Tip: Try searching for:")
        print("   - Stock names: 'HDFC', 'Reliance', 'TCS', 'Infosys'")
        print("   - Keywords: 'Bank', 'Pharma', 'Tata', 'Auto'")
        print("   - Or sector names: 'BANKING', 'IT', 'PHARMA', 'AUTO', etc.")
        print("\n   Type 'sectors' to see all available sectors!")

    return results

def list_sectors():
    """📊 Display all available sectors"""
    print("\n" + "="*70)
    print("📊 AVAILABLE INDIAN STOCK SECTORS")
    print("="*70)
    for sector, stocks in INDIAN_STOCKS.items():
        print(f"  {sector:<15} - {len(stocks)} stocks")
    print(f"\n  Total: {len(ALL_INDIAN_STOCKS)} stocks across {len(INDIAN_STOCKS)} sectors")
    print("="*70)

def list_stocks_by_sector(sector):
    """📈 List all stocks in a specific sector"""
    sector = sector.upper()
    if sector in INDIAN_STOCKS:
        print(f"\n{'='*70}")
        print(f"📈 {sector} SECTOR - {len(INDIAN_STOCKS[sector])} STOCKS")
        print(f"{'='*70}")
        print(f"{'Ticker':<20} {'Company Name'}")
        print("-" * 70)
        for ticker, name in INDIAN_STOCKS[sector].items():
            print(f"{ticker:<20} {name}")
        print("="*70)
    else:
        print(f"❌ Sector '{sector}' not found.")
        list_sectors()

def get_stock_info(ticker):
    """Get basic stock information"""
    ticker = ticker.upper()
    if ticker in ALL_INDIAN_STOCKS:
        sector = next((s for s, stocks in INDIAN_STOCKS.items() if ticker in stocks), 'Unknown')
        return {'ticker': ticker, 'name': ALL_INDIAN_STOCKS[ticker], 'sector': sector}
    return None

def fetch_stock_data(ticker, period='1y', interval='1d'):
    """📥 Fetch stock data from Yahoo Finance"""
    print(f"\n🔹 Fetching data for {ticker}...")
    try:
        df = yf.download(ticker, period=period, interval=interval, progress=False)
        if df.empty:
            print("❌ No data returned")
            return None
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        df.columns = [col.lower() for col in df.columns]
        print(f"✅ Fetched {len(df)} rows from {df.index[0].date()} to {df.index[-1].date()}")
        return df
    except Exception as e:
        print(f"❌ Error fetching data: {e}")
        return None

def add_technical_indicators(df):
    """📊 Add technical indicators to dataframe"""
    print("\n📊 Adding technical indicators...")
    df = df.copy()

    # Moving Averages
    for period in [20, 50, 200]:
        if len(df) >= period:
            df[f'sma_{period}'] = df['close'].rolling(window=period).mean()
            df[f'ema_{period}'] = df['close'].ewm(span=period, adjust=False).mean()

    # RSI
    if len(df) >= 14:
        delta = df['close'].diff()
        gain = delta.where(delta > 0, 0).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / (loss + 1e-10)
        df['rsi_14'] = 100 - (100 / (1 + rs))

    # MACD
    exp1 = df['close'].ewm(span=12, adjust=False).mean()
    exp2 = df['close'].ewm(span=26, adjust=False).mean()
    df['macd'] = exp1 - exp2
    df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()

    # Bollinger Bands
    if len(df) >= 20:
        sma = df['close'].rolling(window=20).mean()
        std = df['close'].rolling(window=20).std()
        df['bb_upper'] = sma + (2 * std)
        df['bb_lower'] = sma - (2 * std)

    df.dropna(inplace=True)
    print(f"✅ Added indicators. Final dataset: {len(df)} rows")
    return df

def generate_signals(df):
    """🎯 Generate trading signals"""
    signals = {'timestamp': df.index[-1], 'close': df['close'].iloc[-1], 'signals': []}

    if 'rsi_14' in df.columns:
        rsi = df['rsi_14'].iloc[-1]
        if rsi < 30:
            signals['signals'].append(f"🟢 RSI ({rsi:.1f}): OVERSOLD - Buy signal")
        elif rsi > 70:
            signals['signals'].append(f"🔴 RSI ({rsi:.1f}): OVERBOUGHT - Sell signal")
        else:
            signals['signals'].append(f"⚪ RSI ({rsi:.1f}): NEUTRAL")

    if 'macd' in df.columns and 'macd_signal' in df.columns:
        if df['macd'].iloc[-1] > df['macd_signal'].iloc[-1]:
            signals['signals'].append(f"🟢 MACD: Bullish (above signal)")
        else:
            signals['signals'].append(f"🔴 MACD: Bearish (below signal)")

    return signals

def analyze_stock(ticker, period='1y', interval='1d', plot=True):
    """📈 Complete stock analysis"""
    ticker = ticker.upper()
    info = get_stock_info(ticker)

    print(f"\n{'='*70}")
    print(f"📈 ANALYZING: {ticker}")
    if info:
        print(f"Company: {info['name']}")
        print(f"Sector: {info['sector']}")
    print(f"{'='*70}")

    df = fetch_stock_data(ticker, period, interval)
    if df is None:
        return None

    df = add_technical_indicators(df)
    if df is None or len(df) == 0:
        return None

    # Statistics
    print(f"\n{'='*70}")
    print("📊 SUMMARY")
    print(f"{'='*70}")
    print(f"Current Price: ₹{df['close'].iloc[-1]:.2f}")
    print(f"High: ₹{df['high'].max():.2f}")
    print(f"Low: ₹{df['low'].min():.2f}")
    print(f"Avg Volume: {df['volume'].mean():,.0f}")

    # Signals
    signals = generate_signals(df)
    print(f"\n🎯 TRADING SIGNALS")
    print("-" * 70)
    for sig in signals['signals']:
        print(f"  {sig}")
    print("="*70)

    if plot:
        plot_chart(df, ticker)

    return df

def plot_chart(df, ticker):
    """📊 Plot interactive chart"""
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                        subplot_titles=[f'{ticker} - Price & Moving Averages', 'Volume', 'RSI'],
                        vertical_spacing=0.05, row_heights=[0.5, 0.25, 0.25])

    # Candlestick
    fig.add_trace(go.Candlestick(x=df.index, open=df['open'], high=df['high'],
                                  low=df['low'], close=df['close'], name='Price'), row=1, col=1)

    # Moving Averages
    for ma, color in [('sma_20', 'orange'), ('sma_50', 'blue'), ('sma_200', 'red')]:
        if ma in df.columns:
            fig.add_trace(go.Scatter(x=df.index, y=df[ma], mode='lines',
                                    name=ma.upper(), line=dict(color=color, width=1)), row=1, col=1)

    # Volume
    colors = ['green' if df['close'].iloc[i] >= df['open'].iloc[i] else 'red' for i in range(len(df))]
    fig.add_trace(go.Bar(x=df.index, y=df['volume'], name='Volume',
                        marker_color=colors, opacity=0.7), row=2, col=1)

    # RSI
    if 'rsi_14' in df.columns:
        fig.add_trace(go.Scatter(x=df.index, y=df['rsi_14'], mode='lines',
                                name='RSI', line=dict(color='purple', width=2)), row=3, col=1)
        fig.add_hline(y=70, line_dash="dash", line_color="red", row=3, col=1)
        fig.add_hline(y=30, line_dash="dash", line_color="green", row=3, col=1)

    fig.update_layout(title=f"{ticker} - Technical Analysis", template='plotly_dark',
                     height=900, showlegend=True, hovermode='x unified')
    fig.update_xaxes(rangeslider_visible=False)
    fig.show()


In [5]:
print("\n" + "="*70)
print("🇮🇳 INDIAN STOCK MARKET ANALYSIS TOOL - READY!")
print("="*70)

print("\n📚 Available Functions:")
print("  search_stock('QUERY')              - 🔍 Search for stocks")
print("  list_sectors()                     - 📊 Show all sectors")
print("  list_stocks_by_sector('SECTOR')   - 📈 List stocks in sector")
print("  analyze_stock('TICKER')            - 📊 Full analysis with charts")

#Search
print("\n" + "="*70)
print("🔍 SEARCH FUNCTION")
print("="*70)
search_stock('TCS')

# Sectors
print("\n" + "="*70)
print(" 📊 SECTORS LIST")
print("="*70)
list_sectors()

# Interactive Search Feature
print("\n" + "="*70)
print("🔍 INTERACTIVE SEARCH")
print("="*70)
print("You can search for:")
print("  • Stock names: HDFC, Reliance, TCS, Infosys, Tata")
print("  • Keywords: Bank, Pharma, Auto, Cement")
print("  • Sectors: BANKING, IT, PHARMA, AUTO, ENERGY, FMCG")
print("  • Type 'sectors' to see all sectors")
print("  • Type 'quit' to exit")
print("="*70)

while True:
    try:
        user_input = input("\n🔍 Search: ").strip()

        if user_input.lower() in ['quit', 'exit', 'q']:
            print("\n✅ Exiting search. You can still use search_stock('QUERY') anytime!")
            break

        if user_input.lower() == 'sectors':
            list_sectors()
            continue

        if not user_input:
            print("⚠️ Please enter a search term!")
            continue

        results = search_stock(user_input)

        if results and len(results) > 0:
            # Ask if they want to analyze
            analyze_choice = input("\n💡 Analyze a stock? Enter ticker (or press Enter to continue): ").strip()
            if analyze_choice:
                analyze_stock(analyze_choice.upper(), period='6mo', plot=True)

    except KeyboardInterrupt:
        print("\n\n✅ Search interrupted. You can still use search_stock('QUERY') anytime!")
        break
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "="*70)
print("✅ TOOL READY FOR USE!")
print("="*70)
print("\n💡 You can still use these commands anytime:")
print("  search_stock('HDFC')")
print("  search_stock('BANK')")
print("  list_stocks_by_sector('IT')")
print("  analyze_stock('TCS.NS', period='6mo')")
print("  analyze_stock('RELIANCE.NS', period='1y')")
print("="*70)


🇮🇳 INDIAN STOCK MARKET ANALYSIS TOOL - READY!

📚 Available Functions:
  search_stock('QUERY')              - 🔍 Search for stocks
  list_sectors()                     - 📊 Show all sectors
  list_stocks_by_sector('SECTOR')   - 📈 List stocks in sector
  analyze_stock('TICKER')            - 📊 Full analysis with charts

🔍 SEARCH FUNCTION

🔍 SEARCH RESULTS FOR: 'TCS' - Found 1 matches
Ticker               Company Name
----------------------------------------------------------------------
TCS.NS               Tata Consultancy Services

 📊 SECTORS LIST

📊 AVAILABLE INDIAN STOCK SECTORS
  NIFTY_50        - 50 stocks
  BANKING         - 12 stocks
  IT              - 9 stocks
  PHARMA          - 9 stocks
  AUTO            - 9 stocks
  FMCG            - 9 stocks
  ENERGY          - 10 stocks
  METALS          - 8 stocks
  INFRA           - 7 stocks
  TELECOM         - 2 stocks

  Total: 83 stocks across 10 sectors

🔍 INTERACTIVE SEARCH
You can search for:
  • Stock names: HDFC, Reliance, TCS, Inf


🔍 Search: quit

✅ Exiting search. You can still use search_stock('QUERY') anytime!

✅ TOOL READY FOR USE!

💡 You can still use these commands anytime:
  search_stock('HDFC')
  search_stock('BANK')
  list_stocks_by_sector('IT')
  analyze_stock('TCS.NS', period='6mo')
  analyze_stock('RELIANCE.NS', period='1y')
